<a href="https://colab.research.google.com/github/Avijit-Panda/CAPSTONE/blob/learning-algos-come-here/model_spifog_integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import numpy as np

In [ ]:
!pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 51.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
sfreq=128


In [ ]:
from scipy.io import loadmat
import mne
data=loadmat("/content/drive/MyDrive/dataset/ADHD_part1/v3p.mat")
eeg_data = data['v3p']
#we transpose as we require the data to be in the form of (num_channels,num_samles)
eeg_data=eeg_data.transpose()
eeg_data.shape
ch_names = ['Fz', 'Cz', 'Pz', 'C3', 'T3', 'C4', 'T4', 'Fp1', 'Fp2', 'F3', 'F4', 'F7', 'F8', 'P3', 'P4', 'T5', 'T6', 'O1', 'O2']  # List of channel names
sfreq = 128  # Sampling frequency also provided in dataset

# Create MNE info structure
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types='eeg')

# Create MNE Raw object
raw = mne.io.RawArray(data=eeg_data, info=info)

In [ ]:
from glob import glob
import os
import mne
import numpy as np
import pandas
import matplotlib.pyplot as plt

In [ ]:
patient_file_path=glob("/content/drive/MyDrive/dataset/ADHD_part1/*.mat")
control_file_path=glob("/content/drive/MyDrive/dataset/Control_part1/*.mat")
print(len(patient_file_path))
print((len(control_file_path)))

In [ ]:
def read_data(file_path):
  data=mne.io.read_raw_edf(file_path,preload=True)
  data.set_eeg_reference()
  data.filter(l_freq=2,h_freq=30,fir_design="firwin")
  ica = mne.preprocessing.ICA(n_components=19, random_state=69,method='fastica')#n_components shouldnt be more htan the no of labels so for trial ill use 19
  ica.fit(data)
  theta_band = (4, 8)   # Theta band (4-8 Hz)
  beta_band = (13, 30)  # Beta band (13-30 Hz)

  # Compute the power spectral density (PSD) for each IC
  psd = ica.get_sources(data, start=0, stop=data.times[-1]).get_data()
  psd = np.abs(np.fft.fft(psd))**2

    # Define frequency values and indices within the theta and beta bands
  freqs = np.fft.fftfreq(data.n_times, d=1/data.info['sfreq'])
  theta_indices = np.where((freqs >= theta_band[0]) & (freqs <= theta_band[1]))[0]
  beta_indices = np.where((freqs >= beta_band[0]) & (freqs <= beta_band[1]))[0]
  theta_metrics = np.mean(psd[:, theta_indices], axis=1)
  beta_metrics = np.mean(psd[:, beta_indices], axis=1)
  theta_std=np.std(psd[:,theta_indices],axis=1)
  beta_std=np.std(psd[:,beta_indices],axis=1)
  threshold_multiplier=2#statistical analysis, larger than this mustbe artifacts
  theta_threshold = threshold_multiplier*theta_std
  beta_threshold = threshold_multiplier*beta_std
  theta_artifact_ICs = np.where(theta_metrics > theta_threshold)[0]
  beta_artifact_ICs = np.where(beta_metrics > beta_threshold)[0]

  artifact_ICs = np.union1d(theta_artifact_ICs, beta_artifact_ICs)

  ica.exclude = artifact_ICs


  cleaned_data = data.copy()
  ica.apply(cleaned_data)
  epochs=mne.make_fixed_length_epochs(cleaned_data,duration=3,overlap=1)
  array=epochs.get_data()
  return array

In [ ]:
%%capture
#reading from control and adhd files
control_epoch_array=[read_data(i) for i in control_file_path]
patient_epoch_array=[read_data(i) for i in patient_file_path]

In [ ]:
control_epoch_labels=[len(i)*[0] for i in control_epoch_array]
patient_epoch_labels=[len(i)*[1] for i in patient_epoch_array]
len(control_epoch_labels),len(patient_epoch_labels)

In [ ]:
data_list=control_epoch_array+patient_epoch_array
label_list=control_epoch_labels+patient_epoch_labels

In [ ]:
group_list=[[i]*len(j) for i,j in enumerate(data_list)]

In [ ]:
# vstack() is used to stack or concate the sequence of given arrays vertically(row-wise).
data_array=np.vstack(data_list)
# hstack() stacks arrays in sequence horizontally (column wise)
label_array=np.hstack(label_list)
group_array=np.hstack(group_list)
print(data_array.shape,label_array.shape,group_array.shape)

In [ ]:
import scipy.io
import mne

def read_data(file_path):
    # Load .mat file
    mat_data = scipy.io.loadmat(file_path)

    # Extract EEG data
    eeg_data = mat_data[(file_path.split("/")[-1]).split(".")[0]]
    eeg_data=eeg_data.transpose()

    # Define channel names
    ch_names = ['Fz', 'Cz', 'Pz', 'C3', 'T3', 'C4', 'T4', 'Fp1', 'Fp2', 'F3', 'F4', 'F7', 'F8', 'P3', 'P4', 'T5', 'T6', 'O1', 'O2']
    # Define sampling frequency
    sfreq = 128
    # Create MNE info structure
    info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types='eeg')
    # Create MNE Raw object
    raw = mne.io.RawArray(data=eeg_data, info=info)
    # Set EEG reference
    # now trying with average
    raw.set_eeg_reference(ref_channels='average')
    # Apply bandpass filter
    # Delta (0.5 - 4 Hz): Associated with deep sleep and certain cognitive processes.
    # Theta (4 - 8 Hz): Related to memory processes, attention, and some meditative states.
    # Alpha (8 - 13 Hz): Most prominent in relaxed states with eyes closed, often related to a resting state or mental relaxation.
    # Beta (13 - 30 Hz): Associated with active cognitive processing, alertness, and concentration.
    # Gamma (30 - 100 Hz): Often observed during complex cognitive tasks and sensory integration.
    raw.filter(l_freq=4, h_freq=12)
    # Create fixed-length epochs
    duration = 5  # Epoch duration in seconds
    overlap = 0.5  # Epoch overlap in seconds
    #events = mne.make_fixed_length_events(raw, duration=duration, overlap=1)
    #epochs = mne.Epochs(raw, events, tmin=0, tmax=duration, baseline=None, preload=True)
    # an epoch refers to a segment of continuous EEG data that is extracted based on specific criteria, such as a fixed duration or the occurrence of an event. It is a way to divide the continuous EEG signal into smaller, temporally discrete segments for analysis.
    epochs=mne.make_fixed_length_epochs(raw,duration=5,overlap=overlap)
    array=epochs.get_data()
    # Get data array from epochs
    array = epochs.get_data()
    return array


In [ ]:
!pip install snntorch

In [ ]:
import snntorch as snn


In [ ]:
class LIFNeuron(nn.Module):
    def __init__(self, dt):
        super(LIFNeuron, self).__init__()
        self.dt = dt
        self.tau_m = 20.0  # Membrane time constant
        self.v_reset = 0.0  # Reset potential
        self.v_threshold = 1.0  # Spike threshold
        self.refractory_period = 5.0  # Refractory period

        # Neuron state variables
        self.v = self.register_buffer("v", torch.zeros(1))
        self.refractory = self.register_buffer("refractory", torch.zeros(1))

    def forward(self, I):
        if not self.refractory:
            self.v += (-self.v + I * self.tau_m) * self.dt / self.tau_m

            if self.v >= self.v_threshold:
                self.v = self.v_reset
                self.refractory = self.refractory_period
                return torch.tensor(1.0)
        else:
            self.refractory -= self.dt

        return torch.tensor(0.0)

# Define the SNN model with LIF neurons
class LIFModel(nn.Module):
    def __init__(self, dt):
        super(LIFModel, self).__init__()

        self.lif1 = LIFNeuron(dt=dt)
        self.lif2 = LIFNeuron(dt=dt)

    def forward(self, x):
        spike1 = self.lif1(x)
        spike2 = self.lif2(spike1)
        return spike2

In [ ]:
import copy

In [ ]:
class SpiFoGLearning:
    def __init__(self, population_size, generations, elite_fraction, dt):
        self.population_size = population_size
        self.generations = generations
        self.elite_fraction = elite_fraction
        self.dt = dt
        self.population = self.initialize_population()

    def initialize_population(self):
        population = []
        for _ in range(self.population_size):
            lif_model = LIFModel(dt=self.dt)
            population.append(lif_model)
        return population

    def calculate_fitness(actual_spike_times, desired_spike_times):
      squared_errors = (actual_spike_times - desired_spike_times)**2
      mean_squared_error = np.mean(squared_errors)
      return mean_squared_error


    def crossover(parent1, parent2):
        child = copy.deepcopy(parent1)
        crossover_point = np.random.randint(0, len(child.synaptic_weights))

        # go9ing to do crossover of synaptic weights and delays
        child.synaptic_weights[crossover_point:] = parent2.synaptic_weights[crossover_point:]
        child.synaptic_delays[crossover_point:] = parent2.synaptic_delays[crossover_point:]

        return child
    def mutate(snn):
        mutated_snn = copy.deepcopy(snn)
        mutation_rate = 0.1

        # do for i mutation to synaptic weights and delays
        for i in range(len(mutated_snn.synaptic_weights)):
            if np.random.rand() < mutation_rate:
                mutated_snn.synaptic_weights[i] = np.random.uniform(-1, 1)
                mutated_snn.synaptic_delays[i] = np.random.uniform(0, 10)

        return mutated_snn


    def select_elite_population(self):
        fitness_scores = [self.calculate_fitness(model) for model in self.population]
        elite_indices = torch.argsort(fitness_scores)[-int(self.elite_fraction * len(self.population)):]
        elite_population = [self.population[i] for i in elite_indices]
        return elite_population



    def spifog_learning(self, input_data, desired_spike_times):
        for generation in range(self.generations):
            elite_population = self.select_elite_population()

            new_population = []
            while len(new_population) < self.population_size:
                parent1 = torch.random.choice(elite_population)
                parent2 = torch.random.choice(elite_population)
                child = self.crossover(parent1, parent2)
                mutated_child = self.mutate(child)
                new_population.append(mutated_child)

            self.population = new_population

        best_model = self.select_best_model()
        return best_model

# # Define the LIF neuron model
# class LIFNeuron(nn.Module):
#     def __init__(self, dt):
#         super(LIFNeuron, self).__init__()
#         self.dt = dt
#         self.tau_m = 20.0  # Membrane time constant
#         self.v_reset = 0.0  # Reset potential
#         self.v_threshold = 1.0  # Spike threshold
#         self.refractory_period = 5.0  # Refractory period

#         # Neuron state variables
#         self.v = self.register_buffer("v", torch.zeros(1))
#         self.refractory = self.register_buffer("refractory", torch.zeros(1))

#     def forward(self, I):
#         if not self.refractory:
#             self.v += (-self.v + I * self.tau_m) * self.dt / self.tau_m

#             if self.v >= self.v_threshold:
#                 self.v = self.v_reset
#                 self.refractory = self.refractory_period
#                 return torch.tensor(1.0)
#         else:
#             self.refractory -= self.dt

#         return torch.tensor(0.0)

# # Define the SNN model with LIF neurons
# class LIFModel(nn.Module):
#     def __init__(self, dt):
#         super(LIFModel, self).__init__()

#         self.lif1 = LIFNeuron(dt=dt)
#         self.lif2 = LIFNeuron(dt=dt)

#     def forward(self, x):
#         spike1 = self.lif1(x)
#         spike2 = self.lif2(spike1)
#         return spike2

# # Example usage
# if __name__ == "__main__":
#     dt = 1.0  # Time step (ms)
#     spifog = SpiFoGLearning(population_size=50, generations=100, elite_fraction=0.2, dt=dt)
#     input_data = torch.tensor([0.2])
#     desired_spike_times = torch.tensor([1.0])

#     best_model = spifog.spifog_learning(input_data, desired_spike_times)

In [ ]:
import torch
import torch.nn as nn
import snntorch as snn

# Define the model
class FeedforwardSNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # Parameters
        self.beta = 0.99

        # Layers
        self.fc1 = nn.Linear(self.input_size, self.hidden_size)
        self.lif1 = snn.Leaky(beta=self.beta)
        self.fc2 = nn.Linear(self.hidden_size, self.output_size)
        self.lif2 = snn.Leaky(beta=self.beta)

    def forward(self, x):
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()

        spk1, mem1 = self.lif1(self.fc1(x), mem1)
        spk2, mem2 = self.lif2(self.fc2(spk1), mem2)

        return spk2, mem2

In [ ]:
dt = 1.0  # Time step (ms)
spifog = SpiFoGLearning(population_size=50, generations=100, elite_fraction=0.2, dt=dt)
input_data = torch.tensor([0.2])
desired_spike_times = torch.tensor([1.0])
best_model = spifog.spifog_learning(input_data, desired_spike_times)